In [20]:
import pandas as pd

In [21]:
df = pd.read_csv('rawdata/yearly_data2.csv')
df.head()

,AQSID,SiteName,Status,EPARegion,Latitude,Longitude,Elevation,GMTOffset,CountryCode,StateName,...,OZONE,OZONE_Unit,NO2,NO2_Unit,CO,CO_Unit,SO2,SO2_Unit,PM10,PM10_Unit
0,250250002,BOSTON-KENMORE,Active,R1,42.348900,-71.097702,0.0,-5.0,US,MA,...,NaN,NaN,18.0,PPB,NaN,NaN,1.0,PPB,NaN,NaN
1,250250044,Boston-Von Hillern,Active,R1,42.325197,-71.056056,4.0,-5.0,US,MA,...,NaN,NaN,20.0,PPB,0.37,PPM,NaN,NaN,NaN,NaN
2,250250042,Boston - Roxbury,Active,R1,42.329399,-71.082497,0.0,-5.0,US,MA,...,16.0,PPB,15.0,PPB,0.37,PPM,1.0,PPB,NaN,NaN
3,250250043,Boston - North End,Inactive,R1,42.363100,-71.054200,0.0,-5.0,US,MA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,250250041,Boston - Long Island,Inactive,R1,42.317200,-70.966700,0.0,-5.0,US,MA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Drop sites with no data

In [22]:
# check if these sites are active (inactive -> nan values for all columns)
df[df["SiteName"]=="Boston - Long Island"]["Status"].value_counts()
df[df["SiteName"]=="Boston - North End"]["Status"].value_counts()

Status
Inactive    46
Name: count, dtype: int64

In [23]:
df = df[df["SiteName"]!="Boston - Long Island"]
df = df[df["SiteName"]!="Boston - North End"]
df["SiteName"].value_counts()

SiteName
BOSTON-KENMORE        46
Boston-Von Hillern    46
Boston - Roxbury      46
BOSTON-Chinatown       7
Name: count, dtype: int64

## Create dataframe for analysis
Some columns are not needed for analysis, so they are dropped.

In [24]:
df = df.drop(["StateName", "CountryCode", "DataSource", "ReportingArea_PipeDelimited", "ValidTime", "GMTOffset", "Elevation", "Longitude", "Latitude", "EPARegion", "AQSID", "Status"], axis=1)

In [25]:
df.head()

,SiteName,ValidDate,OZONE_AQI,PM10_AQI,PM25_AQI,NO2_AQI,OZONE_Measured,PM10_Measured,PM25_Measured,NO2_Measured,...,OZONE,OZONE_Unit,NO2,NO2_Unit,CO,CO_Unit,SO2,SO2_Unit,PM10,PM10_Unit
0,BOSTON-KENMORE,01/01/20,NaN,NaN,NaN,17.0,0,0,0,1,...,NaN,NaN,18.0,PPB,NaN,NaN,1.0,PPB,NaN,NaN
1,Boston-Von Hillern,01/01/20,NaN,NaN,22.0,19.0,0,0,1,1,...,NaN,NaN,20.0,PPB,0.37,PPM,NaN,NaN,NaN,NaN
2,Boston - Roxbury,01/01/20,17.0,NaN,20.0,14.0,1,0,1,1,...,16.0,PPB,15.0,PPB,0.37,PPM,1.0,PPB,NaN,NaN
5,BOSTON-KENMORE,02/01/20,NaN,NaN,NaN,36.0,0,0,0,1,...,NaN,NaN,38.0,PPB,NaN,NaN,1.0,PPB,NaN,NaN
6,Boston-Von Hillern,02/01/20,NaN,NaN,44.0,36.0,0,0,1,1,...,NaN,NaN,38.0,PPB,0.41,PPM,NaN,NaN,NaN,NaN


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 145 entries, 0 to 235
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SiteName        145 non-null    object 
 1   ValidDate       145 non-null    object 
 2   OZONE_AQI       46 non-null     float64
 3   PM10_AQI        5 non-null      float64
 4   PM25_AQI        133 non-null    float64
 5   NO2_AQI         137 non-null    float64
 6   OZONE_Measured  145 non-null    int64  
 7   PM10_Measured   145 non-null    int64  
 8   PM25_Measured   145 non-null    int64  
 9   NO2_Measured    145 non-null    int64  
 10  PM25            114 non-null    float64
 11  PM25_Unit       114 non-null    object 
 12  OZONE           85 non-null     float64
 13  OZONE_Unit      85 non-null     object 
 14  NO2             131 non-null    float64
 15  NO2_Unit        131 non-null    object 
 16  CO              81 non-null     float64
 17  CO_Unit         81 non-null     object 


## Isolate Features that we are interested in

In [27]:
features_interested = ['SiteName', 'ValidDate','OZONE_AQI','PM25_AQI', 'NO2_AQI', 'PM10_AQI']
data = df[features_interested]
data.head()

,SiteName,ValidDate,OZONE_AQI,PM25_AQI,NO2_AQI,PM10_AQI
0,BOSTON-KENMORE,01/01/20,NaN,NaN,17.0,NaN
1,Boston-Von Hillern,01/01/20,NaN,22.0,19.0,NaN
2,Boston - Roxbury,01/01/20,17.0,20.0,14.0,NaN
5,BOSTON-KENMORE,02/01/20,NaN,NaN,36.0,NaN
6,Boston-Von Hillern,02/01/20,NaN,44.0,36.0,NaN


In [31]:
# fill null value with mean value
data['OZONE_AQI'].fillna(data['OZONE_AQI'].mean(), inplace=True)
data['PM25_AQI'].fillna(data['PM25_AQI'].mean(), inplace=True)
data['NO2_AQI'].fillna(data['NO2_AQI'].mean(), inplace=True)
data['PM10_AQI'].fillna(data['PM10_AQI'].mean(), inplace=True)

data.head()

/var/folders/37/5r4fjllx115gd2m3kmqy_02m0000gn/T/ipykernel_76292/4272893825.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['OZONE_AQI'].fillna(data['OZONE_AQI'].mean(), inplace=True)
/var/folders/37/5r4fjllx115gd2m3kmqy_02m0000gn/T/ipykernel_76292/4272893825.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['PM25_AQI'].fillna(data['PM25_AQI'].mean(), inplace=True)
/var/folders/37/5r4fjllx115gd2m3kmqy_02m0000gn/T/ipykernel_76292/4272893825.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pand

,SiteName,ValidDate,OZONE_AQI,PM25_AQI,NO2_AQI,PM10_AQI
0,BOSTON-KENMORE,2020-01-01,24.934783,29.691729,17.0,16.0
1,Boston-Von Hillern,2020-01-01,24.934783,22.000000,19.0,16.0
2,Boston - Roxbury,2020-01-01,17.000000,20.000000,14.0,16.0
3,BOSTON-KENMORE,2020-02-01,24.934783,29.691729,36.0,16.0
4,Boston-Von Hillern,2020-02-01,24.934783,44.000000,36.0,16.0


## Sort by date and aggregate by month

In [32]:
# Sort the DataFrame by the 'ValidDate' column in ascending order
data['ValidDate'] = pd.to_datetime(data['ValidDate'])
data.sort_values(by='ValidDate', ascending=True, inplace=True)

# If you want to reset the index after sorting
data.reset_index(drop=True, inplace=True)

agg_data = data.groupby('SiteName').mean().drop("ValidDate", axis=1)
agg_data

/var/folders/37/5r4fjllx115gd2m3kmqy_02m0000gn/T/ipykernel_76292/1397064884.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ValidDate'] = pd.to_datetime(data['ValidDate'])
/var/folders/37/5r4fjllx115gd2m3kmqy_02m0000gn/T/ipykernel_76292/1397064884.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.sort_values(by='ValidDate', ascending=True, inplace=True)


,OZONE_AQI,PM25_AQI,NO2_AQI,PM10_AQI
SiteName,,,,
BOSTON-Chinatown,24.934783,44.857143,10.919708,16.0
BOSTON-KENMORE,24.934783,30.367767,11.717391,16.0
Boston - Roxbury,24.934783,25.152174,10.260870,16.0
Boston-Von Hillern,24.934783,31.247466,10.780863,16.0


## Assign different sites to neighborhoods that we are analyzing
(TODO)

In [33]:
neighborhood_insterested = ["Allston", "Back Bay", "Bay Village", "Brighton", "Charlestown", "Dorchester", "East Boston", "Fenway", "Harbor Islands", "Hyde Park", "Jamaica Plain", "Leather District", "Longwood Medical Area", "Mattapan", "Mission Hill", "North End", "Roslindale", "Roxbury", "South Boston", "South Boston Waterfront", "South End", "West End", "West Roxbury"]
new_df = pd.DataFrame({"Name": neighborhood_insterested})
new_df.head()

,Name
0,Allston
1,Back Bay
2,Bay Village
3,Brighton
4,Charlestown
